# Azure AI Foundry

<center><img src="../../../images/Azure-AI-Foundry_1600x900.jpg" alt="Azure AI Foundry" width="600">

## Laboratório 5 - RAG (Retrieval-Augmented Generation)

Neste laboratório iremos implementar um sistema RAG (Retrieval-Augmented Generation) usando Azure AI Search e Azure OpenAI. O RAG é uma técnica que combina recuperação de informações com geração de texto, permitindo que os modelos de linguagem acessem conhecimentos específicos externos ao seu treinamento.

Vamos explorar:
1. **Consulta sem RAG**: Como a LLM responde sem conhecimento adicional
2. **Configuração do Azure AI Search**: Preparação do índice de busca vetorial
3. **Implementação do RAG**: Integração da busca com a geração de respostas
4. **Comparação de resultados**: Demonstração da diferença entre respostas com e sem RAG

O primeiro passo é a validação da configuração das variáveis de ambiente no arquivo `.env` presente na raiz do repositório.

Preencha os valores das variáveis de acordo com o solicitado, incluindo as credenciais do Azure AI Search.

### Exercício 1 - Configuração e Importação de Bibliotecas

Vamos realizar a importação das bibliotecas necessárias para o laboratório de RAG.

In [ ]:
import json
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery
import numpy as np

load_dotenv(dotenv_path="../../../.env")

Vamos carregar as credenciais em variáveis para facilitar o uso no laboratório.

In [ ]:
# Azure OpenAI Configuration
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")

# Azure AI Search Configuration
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_KEY")
search_index = os.getenv("AZURE_SEARCH_INDEX", "rag-index")

print("Configurações carregadas:")
print(f"Azure OpenAI Endpoint: {azure_endpoint}")
print(f"Azure Search Endpoint: {search_endpoint}")
print(f"Search Index: {search_index}")

Agora vamos inicializar os clientes do Azure OpenAI e Azure AI Search.

In [ ]:
# Initialize Azure OpenAI client
openai_client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version=api_version
)

# Initialize Azure AI Search client
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=search_index,
    credential=AzureKeyCredential(search_key)
)

print("Clientes inicializados com sucesso!")

### Exercício 2 - Consulta Sem RAG (Baseline)

Primeiro, vamos fazer uma pergunta específica sobre um tópico que provavelmente não está no conhecimento base da LLM ou está desatualizado. Isso nos permitirá comparar a qualidade das respostas antes e depois da implementação do RAG.

In [ ]:
# Pergunta específica sobre um tópico que pode não estar no conhecimento da LLM
question = "Quais são as principais funcionalidades do Azure AI Foundry lançadas em 2024?"

def query_without_rag(question):
    """Faz uma consulta direta ao modelo sem usar RAG"""
    response = openai_client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "Você é um assistente útil especializado em tecnologias Azure."},
            {"role": "user", "content": question}
        ],
        max_tokens=500,
        temperature=0.7
    )
    return response.choices[0].message.content

# Consulta sem RAG
print("=== RESPOSTA SEM RAG ===")
response_without_rag = query_without_rag(question)
print(response_without_rag)
print("\n" + "="*50 + "\n")

### Exercício 3 - Preparação de Dados para RAG

Vamos criar alguns documentos de exemplo sobre Azure AI Foundry para simular uma base de conhecimento. Em um cenário real, estes dados viriam de documentação oficial, manuais, ou outros repositórios de conhecimento.

In [ ]:
# Dados de exemplo sobre Azure AI Foundry
sample_documents = [
    {
        "id": "1",
        "title": "Azure AI Foundry - Visão Geral",
        "content": """Azure AI Foundry é uma plataforma unificada para desenvolvimento de aplicações de IA. 
        Lançado em 2024, oferece ferramentas integradas para construir, treinar e implementar modelos de IA. 
        Inclui suporte nativo para RAG, function calling, e integração com Azure AI Search. 
        A plataforma permite colaboração entre equipes e governança centralizada de modelos."""
    },
    {
        "id": "2", 
        "title": "Funcionalidades do Azure AI Foundry 2024",
        "content": """As principais funcionalidades do Azure AI Foundry incluem: 
        1) Model Catalog com mais de 200 modelos pré-treinados
        2) Prompt Flow para orquestração de workflows de IA
        3) Azure AI Search integrado para implementações RAG
        4) Evaluation tools para métricas de qualidade
        5) Responsible AI dashboard para monitoramento ético
        6) Multi-cloud deployment support
        7) Real-time inference endpoints com auto-scaling"""
    },
    {
        "id": "3",
        "title": "RAG no Azure AI Foundry", 
        "content": """O Azure AI Foundry oferece suporte nativo para Retrieval-Augmented Generation (RAG). 
        Permite conectar facilmente com Azure AI Search, Cosmos DB, e outras fontes de dados. 
        Inclui ferramentas visuais para configurar pipelines RAG sem código. 
        Suporta embeddings personalizados e múltiplos tipos de retrieval como híbrido e semântico."""
    },
    {
        "id": "4",
        "title": "Azure AI Search Integration",
        "content": """A integração com Azure AI Search permite busca vetorial avançada com suporte a:
        - Busca híbrida (keyword + semântica)
        - Filtros de metadados
        - Re-ranking semântico 
        - Múltiplos algoritmos de similaridade (cosine, dot product, euclidean)
        - Indexação automática de documentos
        - Suporte a mais de 300 formatos de arquivo"""
    }
]

print(f"Preparados {len(sample_documents)} documentos de exemplo")
for doc in sample_documents:
    print(f"- {doc['title']}")

### Exercício 4 - Geração de Embeddings

Agora vamos gerar embeddings para nossos documentos usando o Azure OpenAI. Os embeddings são representações vetoriais dos documentos que permitem busca semântica.

In [ ]:
def get_embedding(text):
    """Gera embedding para um texto usando Azure OpenAI"""
    response = openai_client.embeddings.create(
        input=text,
        model=embedding_model
    )
    return response.data[0].embedding

# Gerar embeddings para todos os documentos
print("Gerando embeddings para os documentos...")
for doc in sample_documents:
    # Combinar título e conteúdo para o embedding
    full_text = f"{doc['title']} {doc['content']}"
    doc['embedding'] = get_embedding(full_text)
    print(f"✓ Embedding gerado para: {doc['title']}")

print(f"\nEmbeddings gerados! Dimensão: {len(sample_documents[0]['embedding'])}")

### Exercício 5 - Busca Semântica (Simulada)

Como não temos um índice real do Azure AI Search configurado, vamos simular a busca semântica calculando a similaridade entre embeddings. Em um ambiente real, o Azure AI Search faria isso automaticamente.

In [ ]:
def cosine_similarity(vec1, vec2):
    """Calcula a similaridade coseno entre dois vetores"""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

def semantic_search(query, documents, top_k=2):
    """Realiza busca semântica nos documentos"""
    # Gerar embedding da consulta
    query_embedding = get_embedding(query)
    
    # Calcular similaridades
    similarities = []
    for doc in documents:
        similarity = cosine_similarity(query_embedding, doc['embedding'])
        similarities.append({
            'document': doc,
            'similarity': similarity
        })
    
    # Ordenar por similaridade (maior primeiro)
    similarities.sort(key=lambda x: x['similarity'], reverse=True)
    
    return similarities[:top_k]

# Teste da busca semântica
print("=== BUSCA SEMÂNTICA ===")
results = semantic_search(question, sample_documents)

for i, result in enumerate(results, 1):
    doc = result['document']
    similarity = result['similarity']
    print(f"{i}. {doc['title']} (Similaridade: {similarity:.3f})")
    print(f"   Conteúdo: {doc['content'][:100]}...")
    print()

### Exercício 6 - Implementação do RAG

Agora vamos implementar o sistema RAG completo, combinando a busca semântica com a geração de respostas pelo modelo de linguagem.

In [ ]:
def query_with_rag(question, documents):
    """Implementa RAG: busca documentos relevantes e gera resposta baseada no contexto"""
    
    # 1. Recuperação (Retrieval): Buscar documentos relevantes
    search_results = semantic_search(question, documents, top_k=2)
    
    # 2. Construir contexto com os documentos recuperados
    context_parts = []
    for result in search_results:
        doc = result['document']
        context_parts.append(f"Documento: {doc['title']}\nConteúdo: {doc['content']}")
    
    context = "\n\n".join(context_parts)
    
    # 3. Construir prompt com contexto
    system_message = """Você é um assistente especializado em tecnologias Azure. 
    Use APENAS as informações fornecidas no contexto para responder à pergunta. 
    Se a informação não estiver no contexto, diga que não possui essa informação específica.
    Seja preciso e cite informações específicas do contexto quando possível."""
    
    user_message = f"""Contexto:
{context}

Pergunta: {question}

Resposta baseada no contexto fornecido:"""
    
    # 4. Geração (Augmented Generation): Gerar resposta com contexto
    response = openai_client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        max_tokens=500,
        temperature=0.3  # Menor temperatura para respostas mais precisas
    )
    
    return {
        'answer': response.choices[0].message.content,
        'sources': [result['document']['title'] for result in search_results],
        'context': context
    }

# Executar RAG com a mesma pergunta
print("=== RESPOSTA COM RAG ===")
rag_result = query_with_rag(question, sample_documents)

print("Resposta:")
print(rag_result['answer'])
print(f"\nFontes utilizadas: {', '.join(rag_result['sources'])}")
print("\n" + "="*50 + "\n")

### Exercício 7 - Comparação de Resultados

Vamos comparar diretamente as respostas obtidas sem RAG e com RAG para evidenciar as diferenças e benefícios do uso de conhecimento específico.

In [ ]:
print("🔍 ANÁLISE COMPARATIVA: RAG vs Sem RAG")
print("="*60)

print("\n📝 PERGUNTA:")
print(f'"{question}"')

print("\n❌ RESPOSTA SEM RAG (conhecimento limitado):")
print("-" * 50)
print(response_without_rag)

print("\n✅ RESPOSTA COM RAG (conhecimento aumentado):")
print("-" * 50)
print(rag_result['answer'])

print(f"\n📚 FONTES UTILIZADAS NO RAG:")
for source in rag_result['sources']:
    print(f"• {source}")

print("\n💡 BENEFÍCIOS OBSERVADOS DO RAG:")
benefits = [
    "✓ Informações mais específicas e atualizadas",
    "✓ Respostas baseadas em fontes confiáveis",
    "✓ Maior precisão em detalhes técnicos", 
    "✓ Rastreabilidade das informações (sources)",
    "✓ Redução de alucinações do modelo"
]

for benefit in benefits:
    print(benefit)

### Exercício 8 - RAG com Azure AI Search (Exemplo Real)

Embora tenhamos simulado a busca semântica, vamos mostrar como seria a implementação real usando Azure AI Search. Este código demonstra como conectar com um índice real.

In [ ]:
def query_with_azure_ai_search(question, search_client, openai_client):
    """
    Implementação real de RAG usando Azure AI Search
    Nota: Este código requer um índice configurado no Azure AI Search
    """
    try:
        # 1. Gerar embedding da pergunta
        query_embedding = get_embedding(question)
        
        # 2. Criar consulta vetorizada para Azure AI Search
        vector_query = VectorizedQuery(
            vector=query_embedding,
            k_nearest_neighbors=3,  # Top 3 resultados mais similares
            fields="content_vector"  # Campo que contém os embeddings
        )
        
        # 3. Executar busca no Azure AI Search
        search_results = search_client.search(
            search_text=question,  # Busca híbrida: texto + vetorial
            vector_queries=[vector_query],
            top=3,
            include_total_count=True
        )
        
        # 4. Extrair contexto dos resultados
        context_parts = []
        sources = []
        
        for result in search_results:
            context_parts.append(f"Título: {result['title']}\\nConteúdo: {result['content']}")
            sources.append(result['title'])
        
        context = "\\n\\n".join(context_parts)
        
        # 5. Gerar resposta com contexto
        system_message = """Você é um assistente especializado em Azure. 
        Use APENAS as informações do contexto fornecido para responder."""
        
        user_message = f"""Contexto:\\n{context}\\n\\nPergunta: {question}\\n\\nResposta:"""
        
        response = openai_client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            max_tokens=500,
            temperature=0.3
        )
        
        return {
            'answer': response.choices[0].message.content,
            'sources': sources,
            'context': context
        }
        
    except Exception as e:
        return {
            'answer': f"Erro ao conectar com Azure AI Search: {str(e)}",
            'sources': [],
            'context': ""
        }

# Exemplo de uso (comentado pois requer índice configurado)
print("📋 CÓDIGO PARA AZURE AI SEARCH REAL:")
print("# Para usar este código, você precisa:")
print("# 1. Criar um índice no Azure AI Search")
print("# 2. Configurar campos de embedding") 
print("# 3. Indexar seus documentos")
print("# 4. Executar: query_with_azure_ai_search(question, search_client, openai_client)")

# result = query_with_azure_ai_search(question, search_client, openai_client)
# print(result['answer'])

### Exercício 9 - Teste Interativo

Agora você pode testar o sistema RAG com suas próprias perguntas! Experimente diferentes tipos de consultas para ver como o RAG se comporta.

In [ ]:
# Teste suas próprias perguntas aqui!
test_questions = [
    "Como o Azure AI Foundry suporta RAG?",
    "Quantos modelos estão disponíveis no Model Catalog?",
    "Quais são os tipos de busca suportados pelo Azure AI Search?",
    "O que é o Prompt Flow no Azure AI Foundry?"
]

print("🧪 TESTE INTERATIVO - Experimente diferentes perguntas:")
print("="*60)

for test_q in test_questions:
    print(f"\n❓ Pergunta: {test_q}")
    
    # Resposta sem RAG
    no_rag = query_without_rag(test_q)
    print(f"❌ Sem RAG: {no_rag[:150]}...")
    
    # Resposta com RAG
    with_rag = query_with_rag(test_q, sample_documents)
    print(f"✅ Com RAG: {with_rag['answer'][:150]}...")
    print(f"📚 Fontes: {', '.join(with_rag['sources'])}")
    print("-" * 40)

print("\n💡 Experimente criar suas próprias perguntas modificando a lista 'test_questions'!")

## Conclusão e Próximos Passos

Neste laboratório exploramos o conceito e implementação de RAG (Retrieval-Augmented Generation), demonstrando como:

### ✅ O que aprendemos:
1. **Diferença fundamental**: Como RAG melhora significativamente a qualidade das respostas
2. **Embedding Generation**: Conversão de texto em representações vetoriais
3. **Busca Semântica**: Encontrar documentos relevantes usando similaridade de embeddings
4. **Augmented Generation**: Combinar contexto recuperado com geração de texto
5. **Implementação prática**: Código funcional para sistema RAG completo

### 🚀 Benefícios do RAG demonstrados:
- **Conhecimento atualizado**: Acesso a informações específicas e recentes
- **Redução de alucinações**: Respostas baseadas em fontes confiáveis  
- **Rastreabilidade**: Capacidade de identificar fontes das informações
- **Especialização**: Respostas mais precisas sobre tópicos específicos
- **Flexibilidade**: Fácil atualização da base de conhecimento

### 🔧 Para implementação em produção:
1. **Configure Azure AI Search** com índices vetoriais otimizados
2. **Implemente chunking** para documentos grandes (512-1024 tokens)
3. **Use busca híbrida** (keyword + semântica) para melhor recall
4. **Adicione re-ranking** para melhorar a relevância dos resultados
5. **Monitore performance** e ajuste parâmetros (top_k, temperature, etc.)
6. **Implemente cache** para consultas frequentes
7. **Configure governança** para validação de fontes

### 📚 Recursos adicionais:
- [Azure AI Search Vector Search](https://learn.microsoft.com/en-us/azure/search/vector-search-overview)
- [Azure OpenAI RAG Patterns](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/use-your-data)
- [Azure AI Foundry Documentation](https://learn.microsoft.com/en-us/azure/ai-foundry/)
- [RAG Best Practices](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/advanced-usage)